In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import r2_score

# Load the CSV file
data = pd.read_csv('data_ANN.csv')

# Extract features (AT, V, AP, RH) into X
X = data[['AT', 'V', 'AP', 'RH']].values

# Extract target (PE) into y
y = data[['PE']].values

# Normalize input features
X_mean = X.mean(axis=0)
X_std = X.std(axis=0)
X = (X - X_mean) / X_std  # Standardization

# Normalize target values
y_mean = y.mean()
y_std = y.std()
y = (y - y_mean) / y_std  # Standardization

# Define the neural network architecture
input_size = 4    # Number of input neurons (four features)
hidden_size = 5   # Number of neurons in the hidden layer
output_size = 1   # Number of output neurons (one output)

# Initialize weights and biases
np.random.seed(42)
W1 = np.random.randn(input_size, hidden_size)  # Weights for input to hidden layer
b1 = np.zeros((1, hidden_size))                # Bias for hidden layer
W2 = np.random.randn(hidden_size, output_size) # Weights for hidden to output layer
b2 = np.zeros((1, output_size))                # Bias for output layer

# Activation functions
def relu(x):
    return np.maximum(0, x)

def relu_derivative(x):
    return np.where(x > 0, 1, 0)

# Forward propagation
def forward_propagation(X):
    Z1 = np.dot(X, W1) + b1
    A1 = relu(Z1)
    Z2 = np.dot(A1, W2) + b2
    A2 = Z2  # Linear activation for output
    return Z1, A1, Z2, A2

# Loss function
def compute_loss(y_true, y_pred):
    m = y_true.shape[0]
    loss = np.mean((y_true - y_pred) ** 2)  # Mean Squared Error for regression
    return loss

# Backpropagation
def backpropagation(X, y_true, Z1, A1, Z2, A2):
    m = X.shape[0]
    dZ2 = (A2 - y_true)
    dW2 = np.dot(A1.T, dZ2) / m
    db2 = np.sum(dZ2, axis=0, keepdims=True) / m
    dA1 = np.dot(dZ2, W2.T)
    dZ1 = dA1 * relu_derivative(A1)
    dW1 = np.dot(X.T, dZ1) / m
    db1 = np.sum(dZ1, axis=0, keepdims=True) / m
    return dW1, db1, dW2, db2

# Update parameters
def update_parameters(W1, b1, W2, b2, dW1, db1, dW2, db2, learning_rate):
    W1 -= learning_rate * dW1
    b1 -= learning_rate * db1
    W2 -= learning_rate * dW2
    b2 -= learning_rate * db2
    return W1, b1, W2, b2

# Train the neural network
def train(X, y, epochs, learning_rate):
    global W1, b1, W2, b2
    for epoch in range(epochs):
        Z1, A1, Z2, A2 = forward_propagation(X)
        loss = compute_loss(y, A2)
        if(epoch%500==0):
            print(f"Epoch {epoch + 1}/{epochs} - Loss: {loss}")
        dW1, db1, dW2, db2 = backpropagation(X, y, Z1, A1, Z2, A2)
        W1, b1, W2, b2 = update_parameters(W1, b1, W2, b2, dW1, db1, dW2, db2, learning_rate)
    return W1, b1, W2, b2

# Make predictions
def predict(X, W1, b1, W2, b2):
    _, _, _, A2 = forward_propagation(X)
    return A2

# Train the neural network
W1, b1, W2, b2 = train(X, y, epochs=10000, learning_rate=0.01)

# Make predictions
predictions = predict(X, W1, b1, W2, b2)

# Denormalize the predictions
denormalized_predictions = predictions * y_std + y_mean

# Compute R² score
r2 = r2_score(data[['PE']], denormalized_predictions)

print("Predictions:")
print(denormalized_predictions)
print(f"R² Score: {r2}")


C:\Users\pc\AppData\Local\Temp\ipykernel_480\3674205566.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


Epoch 1/10000 - Loss: 7.917091010584459
Epoch 501/10000 - Loss: 0.2811021868676412
Epoch 1001/10000 - Loss: 0.21106769475025144
Epoch 1501/10000 - Loss: 0.17965102432784555
Epoch 2001/10000 - Loss: 0.16202940331332966
Epoch 2501/10000 - Loss: 0.14794802925160014
Epoch 3001/10000 - Loss: 0.13405419915171424
Epoch 3501/10000 - Loss: 0.11971900241924906
Epoch 4001/10000 - Loss: 0.10680548863546531
Epoch 4501/10000 - Loss: 0.0957851564195749
Epoch 5001/10000 - Loss: 0.08694102451927195
Epoch 5501/10000 - Loss: 0.08061546019367904
Epoch 6001/10000 - Loss: 0.07611978671449682
Epoch 6501/10000 - Loss: 0.07299232990249449
Epoch 7001/10000 - Loss: 0.07077003108676015
Epoch 7501/10000 - Loss: 0.06916478937160588
Epoch 8001/10000 - Loss: 0.06802710015569449
Epoch 8501/10000 - Loss: 0.06718622706606904
Epoch 9001/10000 - Loss: 0.06655313136575433
Epoch 9501/10000 - Loss: 0.06609653938598738
Predictions:
[[466.495635  ]
 [444.46374077]
 [484.14953367]
 ...
 [434.39148765]
 [442.80552008]
 [449.0787